## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
import os

file_path = "input.txt"
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

if not os.path.exists(file_path):
    !wget {url}
else:
    print(f"'{file_path}' already exists, skipping download.")


'input.txt' already exists, skipping download.


In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
# here are all the unique characters that occur in this text
# 本文中出现的所有独特字符
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

In [7]:
# For the mapping from string to integer
print("Mapping from characters to integers:")
for ch, i in stoi.items():
    print(f"'{ch}': {i}")

# Add some spacing
print("\n" + "-"*50 + "\n")

# For the mapping from integer to string
print("Mapping from integers to characters:")
for i, ch in itos.items():
    print(f"{i}: '{ch}'")


Mapping from characters to integers:
'
': 0
' ': 1
'!': 2
'$': 3
'&': 4
''': 5
',': 6
'-': 7
'.': 8
'3': 9
':': 10
';': 11
'?': 12
'A': 13
'B': 14
'C': 15
'D': 16
'E': 17
'F': 18
'G': 19
'H': 20
'I': 21
'J': 22
'K': 23
'L': 24
'M': 25
'N': 26
'O': 27
'P': 28
'Q': 29
'R': 30
'S': 31
'T': 32
'U': 33
'V': 34
'W': 35
'X': 36
'Y': 37
'Z': 38
'a': 39
'b': 40
'c': 41
'd': 42
'e': 43
'f': 44
'g': 45
'h': 46
'i': 47
'j': 48
'k': 49
'l': 50
'm': 51
'n': 52
'o': 53
'p': 54
'q': 55
'r': 56
's': 57
't': 58
'u': 59
'v': 60
'w': 61
'x': 62
'y': 63
'z': 64

--------------------------------------------------

Mapping from integers to characters:
0: '
'
1: ' '
2: '!'
3: '$'
4: '&'
5: '''
6: ','
7: '-'
8: '.'
9: '3'
10: ':'
11: ';'
12: '?'
13: 'A'
14: 'B'
15: 'C'
16: 'D'
17: 'E'
18: 'F'
19: 'G'
20: 'H'
21: 'I'
22: 'J'
23: 'K'
24: 'L'
25: 'M'
26: 'N'
27: 'O'
28: 'P'
29: 'Q'
30: 'R'
31: 'S'
32: 'T'
33: 'U'
34: 'V'
35: 'W'
36: 'X'
37: 'Y'
38: 'Z'
39: 'a'
40: 'b'
41: 'c'
42: 'd'
43: 'e'
44: 'f'
45: 'g'
46: '

In [8]:
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print('x=', x)
print('y=', y)
print('\n')

for t in range(block_size):
    print('t=', t)
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

x= tensor([18, 47, 56, 57, 58,  1, 15, 47])
y= tensor([47, 56, 57, 58,  1, 15, 47, 58])


t= 0
when input is tensor([18]) the target: 47
t= 1
when input is tensor([18, 47]) the target: 56
t= 2
when input is tensor([18, 47, 56]) the target: 57
t= 3
when input is tensor([18, 47, 56, 57]) the target: 58
t= 4
when input is tensor([18, 47, 56, 57, 58]) the target: 1
t= 5
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
t= 6
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
t= 7
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [13]:
torch.manual_seed(1000)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y



In [14]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[47, 57, 46,  1, 40, 50, 53, 53],
        [56, 39, 63,  5, 57,  1, 40, 56],
        [ 1, 15, 47, 58, 47, 64, 43, 52],
        [54, 43,  6,  1, 58, 46, 53, 59]])
targets:
torch.Size([4, 8])
tensor([[57, 46,  1, 40, 50, 53, 53, 42],
        [39, 63,  5, 57,  1, 40, 56, 43],
        [15, 47, 58, 47, 64, 43, 52, 10],
        [43,  6,  1, 58, 46, 53, 59,  1]])


In [32]:
print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

----
when input is [58] the target: 46
when input is [58, 46] the target: 43
when input is [58, 46, 43] the target: 1
when input is [58, 46, 43, 1] the target: 43
when input is [58, 46, 43, 1, 43] the target: 62
when input is [58, 46, 43, 1, 43, 62] the target: 58
when input is [58, 46, 43, 1, 43, 62, 58] the target: 56
when input is [58, 46, 43, 1, 43, 62, 58, 56] the target: 43
when input is [1] the target: 45
when input is [1, 45] the target: 43
when input is [1, 45, 43] the target: 52
when input is [1, 45, 43, 52] the target: 58
when input is [1, 45, 43, 52, 58] the target: 50
when input is [1, 45, 43, 52, 58, 50] the target: 43
when input is [1, 45, 43, 52, 58, 50, 43] the target: 1
when input is [1, 45, 43, 52, 58, 50, 43, 1] the target: 30
when input is [46] the target: 43
when input is [46, 43] the target: 1
when input is [46, 43, 1] the target: 47
when input is [46, 43, 1, 47] the target: 52
when input is [46, 43, 1, 47, 52] the target: 52
when input is [46, 43, 1, 47, 52, 52]

In [33]:
print(xb) # our input to the transformer

tensor([[58, 46, 43,  1, 43, 62, 58, 56],
        [ 1, 45, 43, 52, 58, 50, 43,  1],
        [46, 43,  1, 47, 52, 52, 53, 41],
        [ 6,  1, 57, 39, 63,  5, 57, 58],
        [43, 58, 63, 12,  0, 27, 52,  1],
        [ 6,  1, 57, 47, 56,  6,  1, 61],
        [53,  6,  1, 57, 47, 56, 11,  1],
        [ 2,  0, 18, 53, 56,  1, 25, 39],
        [46, 39, 52, 45, 47, 52, 45,  6],
        [35, 47, 58, 46,  1, 39, 50, 50],
        [ 1, 58, 53,  1, 40, 43,  1, 58],
        [61, 46, 53,  5, 57,  1, 50, 47],
        [ 1, 51, 43,  0, 35, 46, 43, 56],
        [27, 25, 17, 27, 10,  0, 31, 50],
        [57, 46, 53, 61, 57,  1, 39, 40],
        [58, 63,  5, 57,  1, 58, 46, 43],
        [56, 39, 63,  1, 44, 53, 56,  1],
        [47, 58,  5, 57,  1, 57, 54, 56],
        [21, 26, 19, 14, 30, 27, 23, 17],
        [ 0,  0, 16, 27, 30, 15, 13, 31],
        [58, 46, 43,  1, 41, 39, 59, 57],
        [ 1, 53, 52, 43,  1, 45, 53, 53],
        [57,  1, 58, 53, 53,  1, 46, 43],
        [26, 21, 33, 31, 10,  0, 2

In [46]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([256, 65])
tensor(4.7544, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [47]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [54]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
#     print(loss.item())
    
print('-'*20)
print(loss.item())

--------------------
2.666409730911255


In [55]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ARWhac! toindilisthipr alily well fthis I as r ovemyorerouthis matofrhe lavefessuladvoms thaven'd y Ind thig ies we:
Car Whan:
Hpout.
Thont fo ind ove l tt:
TE llaclarr.
en th.
WAD win;
WI.
AMyothomo m
AUSI bend ant frrthi's buthareve y s avel,'t,
K: f-m roke sth f tid Ch IUpechadd angoutee, winmerthorouow wsqus,
MI whines, rcothheanoneshe olithel CLIIFrdatit t be w yongayoy.


Wharoputhe steaple abed
DWhin.
Clang-
QUENCacome dir hay modo
ILISust h t ar o ct wis atino g;
P UE:
Whyofa,
RWeallly s
